In [2]:
!pip install simfin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.2 MB/s eta 0:00:00


In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.lines import Line2D
import multiprocessing as mp
import seaborn as sns
import pandas as pd
import numpy as np
import os
import time
from numba import jit
from scipy.stats import norm
from datetime import timedelta
from functools import partial
from itertools import chain, combinations

In [4]:
# Imports from SimFin.
import simfin as sf
from simfin.names import (TOTAL_RETURN, CLOSE, VOLUME, TICKER, PSALES, DATE)
from simfin.utils import BDAYS_PER_YEAR

In [ ]:
sf.set_data_dir('~/simfin_data/')

# SimFin load API key or use free data.
sf.set_api_key('') # Can't push the key to the public repository.

In [6]:
%%time
hub = sf.StockHub(market='us', refresh_days_shareprices=100)
# Download and load the daily share-prices for US stocks.
df_daily_prices = hub.load_shareprices(variant='daily')

Dataset "us-shareprices-daily" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/usr/local/lib/python3.10/dist-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
CPU times: user 44.5 s, sys: 6.52 s, total: 51 s
Wall time: 1min 9s


In [7]:
%%time
df_val_signals = hub.val_signals()

Dataset "us-income-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/usr/local/lib/python3.10/dist-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-balance-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/usr/local/lib/python3.10/dist-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-cashflow-ttm" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/usr/local/lib/python3.10/dist-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Cache-file 'val_signals-77ca1f39.pickle' not on disk.
- Running function val_signals() ... Done!
- Saving cache-file to disk ... Done!
CPU times: user 21.8 s, sys: 5.8 s, total: 27.6 s
Wall time: 38.3 s


In [8]:
# Use the daily "Total Return" series which is the stock-price
# adjusted for stock-splits and reinvestment of dividends.
# This is a Pandas DataFrame in matrix-form where the rows are
# time-steps and the columns are for the individual stock-tickers.
daily_prices = df_daily_prices[TOTAL_RETURN].unstack().T

# Remove rows that have very little data. Sometimes this dataset
# has "phantom" data-points for a few stocks e.g. on weekends.
num_stocks = len(daily_prices.columns)
daily_prices = daily_prices.dropna(thresh=int(0.1 * num_stocks))

# Remove the last row because sometimes it is incomplete.
daily_prices = daily_prices.iloc[0:-1]

# Show it.
daily_prices.head()

Ticker,A,AA,AAC,AACI,AAGH,AAIC,AAL,AAMC,AAME,AAN,...,ZUUS,ZVIA,ZVO,ZVRA,ZVSA,ZWS,ZY,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-10,67.12,28.05,NaN,NaN,0.02,6.75,31.54,17.41,2.36,NaN,...,NaN,NaN,7.68,41.04,NaN,11.73,NaN,15.80,4.17,2.70
2019-01-11,67.58,27.63,NaN,NaN,0.02,6.78,31.30,17.94,2.50,NaN,...,NaN,NaN,7.68,42.08,NaN,11.63,NaN,15.87,4.12,2.70
2019-01-14,66.98,27.87,NaN,NaN,NaN,6.82,31.21,17.98,2.40,NaN,...,NaN,NaN,7.95,43.20,NaN,11.36,NaN,15.93,3.88,2.71
2019-01-15,67.79,27.97,NaN,NaN,0.02,6.85,31.59,17.60,2.57,NaN,...,NaN,NaN,7.90,40.80,NaN,11.39,NaN,16.16,4.00,2.79
2019-01-16,67.25,28.05,NaN,NaN,NaN,6.94,32.32,16.60,2.40,NaN,...,NaN,NaN,7.78,42.56,NaN,11.40,NaN,16.29,3.92,2.78


In [ ]:
daily_prices.to_csv('prices.csv')